In [4]:
import pickle5 as pkl

df_diag = pkl.load(open("../data/diag_diabetes_final.p",'rb'))
df_notes = pkl.load(open("../data/notes_diabetes_final.p",'rb'))

In [5]:
print(df_diag.shape)
print(df_notes.shape)

(199964, 6)
(399629, 7)


In [8]:
from gensim.models import Word2Vec

list_notes = df_notes['NTOKENS'].to_list()


In [14]:
model = Word2Vec(list_notes, vector_size=300, workers=4, sg=1)


In [16]:
print(model)

model.wv.save_word2vec_format('../data/word2vec_model.bin')


Word2Vec(vocab=52200, vector_size=300, alpha=0.025)


In [25]:
 words = list(model.wv.index_to_key)
print(len(model.wv['response']))

300
